In [1]:
from nipype import Node, Workflow, IdentityInterface, Function, MapNode,SelectFiles, DataSink
from nipype.interfaces import fsl
from nipype.algorithms.modelgen import SpecifyModel
import numpy as np
import os

In [2]:
def gather_subject_info(tsv_filename):
    
    import pandas as pd
    from nipype.interfaces.base import Bunch
    
    trialinfo = pd.read_table(tsv_filename)

    conditions = []
    onsets = []
    durations = []

    for group in trialinfo.groupby('Stimulus'):
        conditions.append(group[0])
        onsets.append(list(group[1].onset)) 
        durations.append(list(group[1].duration))

    subject_info = Bunch(conditions=conditions,
                          onsets=onsets,
                          durations=durations)

    return subject_info

In [3]:
data   = "/media/paradeisios/UUI/fmri/flanker/output/preprocessing"
output = "/media/paradeisios/UUI/fmri/flanker/output/first_level"
brain_mask = "/home/paradeisios/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz"

num_of_subjects = 26
num_of_runs = 2

subject_list = ["{:02d}".format(i) for i in range(1,num_of_subjects+1)]

TR = 2


cont1 = ['Con+',   'T', ['congruent'], [1]]
cont2 = ['Incon+', 'T', ['incongruent'], [1]]
cont3 = ['Con-',   'T', ['congruent'], [-1]]
cont4 = ['Incon-', 'T', ['incongruent'], [-1]]
cont5 = ['Con > Incon', 'T', ['congruent', 'incongruent'], [1, -1]]
cont6 = ['Incon > Con', 'T', ['congruent', 'incongruent'], [-1, 1,]]
contrasts = [cont1,cont2,cont3,cont4,cont5,cont6]

In [4]:
infosource = Node(IdentityInterface(fields=["subject_list"]), name="infosource")
infosource.iterables = [('subject_list', subject_list)]

In [5]:
templates = {"func"       : "{subject_list}/smooth/{subject_list}/smooth_run_*/sub-{subject_list}_task-flanker_run-*_bold_maths_roi_mcf_warp_smooth.nii.gz",
             "real_params": "{subject_list}/realign/{subject_list}/realign_*/sub-{subject_list}_task-flanker_run-*_bold_maths_roi_mcf.nii.gz.par",
             "conditions" : "{subject_list}/conditions/sub-{subject_list}_task-flanker_run-*_events.tsv",
             "outliers"   : "{subject_list}/art/{subject_list}/art_run_*/art.sub-{subject_list}_task-flanker_run-*_bold_maths_roi_mcf_outliers.txt"}

selectfiles = Node(SelectFiles(templates),name="selectfiles")
selectfiles.inputs.base_directory = data

In [6]:
substitutions = []
substitutions += [("_subject_list_{:02d}".format(i), "{:02d}".format(i)) for i in range(1,num_of_subjects+1)]
substitutions += [("_modelestimate{:d}".format(i), "modelestimate_run_{:d}".format(i+1)) for i in range(0,num_of_runs)]
substitutions += [("_modelgen{:d}".format(i), "modelgen_run_{:d}".format(i+1)) for i in range(0,num_of_runs)]
substitutions += [("_level1design{:d}".format(i), "level1design_run_{:d}".format(i+1)) for i in range(0,num_of_runs)]

datasink = Node(DataSink(base_directory=output),name="datasink")
datasink.inputs.substitutions = substitutions
datasink.output = output

In [7]:
getsubjectinfo = MapNode(Function(input_names=["tsv_filename"],output_names=['subject_info'],
                                  function=gather_subject_info),
                                  iterfield = ["tsv_filename"],
                                  name='getsubjectinfo')

mask_functional = MapNode(fsl.maths.ApplyMask(),name="mask_functional",iterfield=["in_file"])
mask_functional.inputs.mask_file = brain_mask

specify_model = MapNode(SpecifyModel(),name="specify_model",
                        iterfield=["functional_runs","subject_info","realignment_parameters","outlier_files"])
specify_model.inputs.input_units = 'secs'
specify_model.inputs.time_repetition = TR
specify_model.inputs.high_pass_filter_cutoff = 100.              
specify_model.inputs.parameter_source = "FSL"

level1design = MapNode(fsl.Level1Design(), name="level1design",iterfield = ["session_info"])# make the model
level1design.inputs.bases = {'dgamma':{'derivs': True}}
level1design.inputs.interscan_interval = TR
level1design.inputs.model_serial_correlations=True
level1design.inputs.contrasts=contrasts

modelgen = MapNode(fsl.FEATModel(),name='modelgen',iterfield = ["fsf_file","ev_files"]) # transform to fsl model

model_threshold = MapNode(fsl.ImageStats(),name="model_threshold",iterfield=["in_file"])
model_threshold.inputs.op_string = "-P 10"

modelestimate = MapNode(fsl.FILMGLS(), name='modelestimate',iterfield=["in_file","design_file","tcon_file","threshold"]) #run the model
modelestimate.inputs.smooth_autocorr=True 
modelestimate.inputs.mask_size=5 

In [8]:
first_level_analysis = Workflow(name="first_level_analysis")

first_level_analysis.connect(infosource, "subject_list", selectfiles, "subject_list")
first_level_analysis.connect(selectfiles, "conditions", getsubjectinfo, "tsv_filename")


first_level_analysis.connect(selectfiles, "func", mask_functional, "in_file")

first_level_analysis.connect(selectfiles, "real_params", specify_model, "realignment_parameters")
first_level_analysis.connect(selectfiles, "outliers", specify_model, "outlier_files")
first_level_analysis.connect(getsubjectinfo,"subject_info",specify_model,"subject_info")
first_level_analysis.connect(mask_functional, "out_file", specify_model, "functional_runs")

first_level_analysis.connect(mask_functional, "out_file", model_threshold, "in_file")

first_level_analysis.connect(specify_model,"session_info",level1design,"session_info")

first_level_analysis.connect(level1design,"fsf_files",modelgen,"fsf_file")
first_level_analysis.connect(level1design,"ev_files",modelgen,"ev_files")

first_level_analysis.connect(mask_functional, "out_file", modelestimate, "in_file")
first_level_analysis.connect(model_threshold, "out_stat", modelestimate, "threshold")
first_level_analysis.connect(modelgen,"design_file",modelestimate,"design_file")
first_level_analysis.connect(modelgen,"con_file",modelestimate,"tcon_file")

first_level_analysis.connect(level1design,"fsf_files",datasink,"level1design.@fsf_files")
first_level_analysis.connect(level1design,"ev_files",datasink,"level1design.@ev_files")

first_level_analysis.connect(modelgen,"design_file",datasink,"model.@design_file")
first_level_analysis.connect(modelgen,"design_cov",datasink,"model.@design_cov")
first_level_analysis.connect(modelgen,"design_image",datasink,"model.@design_matrix")

first_level_analysis.connect(modelestimate,"copes",datasink,"modelestimate.@copes")
first_level_analysis.connect(modelestimate,"varcopes",datasink,"modelestimate.@varcopes")
first_level_analysis.connect(modelestimate,"tstats",datasink,"modelestimate.@tstats")
first_level_analysis.connect(modelestimate,"zstats",datasink,"modelestimate.@zstats")
first_level_analysis.connect(modelestimate,"dof_file",datasink,"modelestimate.@dof_file")

In [9]:
first_level_analysis.write_graph(graph2use='flat', format='png', simple_form=True,
                              dotfilename= os.path.join(output,"first_level_analysis_workflow.dot"))

210815-15:27:46,755 nipype.workflow INFO:
	 Generated workflow graph: /media/paradeisios/UUI/fmri/flanker/output/first_level/first_level_analysis_workflow.png (graph2use=flat, simple_form=True).


'/media/paradeisios/UUI/fmri/flanker/output/first_level/first_level_analysis_workflow.png'

In [10]:
first_level_analysis.run(plugin='MultiProc', plugin_args={'n_procs' : 1})

210815-15:27:46,836 nipype.workflow INFO:
	 Workflow first_level_analysis settings: ['check', 'execution', 'logging', 'monitoring']
210815-15:27:47,247 nipype.workflow INFO:
	 Running in parallel.
210815-15:27:47,256 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 26 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210815-15:27:47,313 nipype.workflow INFO:
	 [Node] Setting-up "first_level_analysis.selectfiles" in "/tmp/tmp22n3eqw7/first_level_analysis/_subject_list_26/selectfiles".
210815-15:27:47,319 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
210815-15:27:47,651 nipype.workflow INFO:
	 [Node] Finished "first_level_analysis.selectfiles".
210815-15:27:49,260 nipype.workflow INFO:
	 [Job 0] Completed (first_level_analysis.selectfiles).
210815-15:27:49,264 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 27 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210815-15:27:49,375 nipype.workflow INFO:
	

210815-15:28:13,285 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 63 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210815-15:28:13,351 nipype.workflow INFO:
	 [Node] Setting-up "first_level_analysis.selectfiles" in "/tmp/tmpjv3a3pja/first_level_analysis/_subject_list_13/selectfiles".
210815-15:28:13,355 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
210815-15:28:13,398 nipype.workflow INFO:
	 [Node] Finished "first_level_analysis.selectfiles".
210815-15:28:15,288 nipype.workflow INFO:
	 [Job 117] Completed (first_level_analysis.selectfiles).
210815-15:28:15,291 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 66 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210815-15:28:15,351 nipype.workflow INFO:
	 [Node] Setting-up "first_level_analysis.selectfiles" in "/tmp/tmp9zdtdg5z/first_level_analysis/_subject_list_12/selectfiles".
210815-15:28:15,355 nipype.workflow INFO:
	 [Node] Running "selectfi

210815-15:28:39,378 nipype.workflow INFO:
	 [Node] Running "_mask_functional0" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /media/paradeisios/UUI/fmri/flanker/output/preprocessing/26/smooth/26/smooth_run_1/sub-26_task-flanker_run-1_bold_maths_roi_mcf_warp_smooth.nii.gz -mas /home/paradeisios/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz /tmp/tmpunms6wxp/first_level_analysis/_subject_list_26/mask_functional/mapflow/_mask_functional0/sub-26_task-flanker_run-1_bold_maths_roi_mcf_warp_smooth_masked.nii.gz
210815-15:28:41,309 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 103 jobs ready. Free memory (GB): 6.54/6.74, Free processors: 0/1.
                     Currently running:
                       * _mask_functional0
210815-15:28:54,570 nipype.workflow INFO:
	 [Node] Finished "_mask_functional0".
210815-15:28:55,324 nipype.workflow INFO:
	 [Job 234] Completed (_mask_functional0).
210815-15:28:55,331 nipype.workflow INFO:
	 [MultiProc]

210815-15:29:51,365 nipype.workflow INFO:
	 [Node] Finished "_mask_functional1".
210815-15:29:51,375 nipype.workflow INFO:
	 [Job 239] Completed (_mask_functional1).
210815-15:29:51,382 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 103 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210815-15:29:51,457 nipype.workflow INFO:
	 [Node] Setting-up "first_level_analysis.mask_functional" in "/tmp/tmpbmnz2_pt/first_level_analysis/_subject_list_25/mask_functional".
210815-15:29:51,475 nipype.workflow INFO:
	 [Node] Setting-up "_mask_functional0" in "/tmp/tmpbmnz2_pt/first_level_analysis/_subject_list_25/mask_functional/mapflow/_mask_functional0".
210815-15:29:51,486 nipype.workflow INFO:
	 [Node] Cached "_mask_functional0" - collecting precomputed outputs
210815-15:29:51,488 nipype.workflow INFO:
	 [Node] "_mask_functional0" found cached.
210815-15:29:51,490 nipype.workflow INFO:
	 [Node] Setting-up "_mask_functional1" in "/tmp/tmpbmnz2_pt/first_level_analysis/_subjec

210815-15:30:33,517 nipype.workflow INFO:
	 [Node] Finished "_getsubjectinfo0".
210815-15:30:35,419 nipype.workflow INFO:
	 [Job 244] Completed (_getsubjectinfo0).
210815-15:30:35,425 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 103 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210815-15:30:35,476 nipype.workflow INFO:
	 [Node] Setting-up "_getsubjectinfo1" in "/tmp/tmp_md7gc63/first_level_analysis/_subject_list_24/getsubjectinfo/mapflow/_getsubjectinfo1".
210815-15:30:35,511 nipype.workflow INFO:
	 [Node] Running "_getsubjectinfo1" ("nipype.interfaces.utility.wrappers.Function")
210815-15:30:35,532 nipype.workflow INFO:
	 [Node] Finished "_getsubjectinfo1".
210815-15:30:37,422 nipype.workflow INFO:
	 [Job 245] Completed (_getsubjectinfo1).
210815-15:30:37,426 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 103 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210815-15:30:37,478 nipype.workflow INFO:
	 [Node] Setting-up "first_level_a

...

210815-19:00:03,493 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/level1design/_subject_list_02/_level1design1/ev_Realign5_0_9.txt -> /media/paradeisios/UUI/fmri/flanker/output/first_level/level1design/02/level1design_run_2/ev_Realign5_0_9.txt
210815-19:00:03,496 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/level1design/_subject_list_02/_level1design1/ev_Realign6_0_10.txt -> /media/paradeisios/UUI/fmri/flanker/output/first_level/level1design/02/level1design_run_2/ev_Realign6_0_10.txt
210815-19:00:03,500 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/model/_subject_list_02/_modelgen0/run0.mat -> /media/paradeisios/UUI/fmri/flanker/output/first_level/model/02/modelgen_run_1/run0.mat
210815-19:00:03,503 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/model/_subject_list_02/_modelgen1/run0.mat -> /media/paradeisios/UUI/fmri/flanker/output

210815-19:00:03,877 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/_subject_list_02/_modelestimate1/varcope6.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/02/modelestimate_run_2/varcope6.nii.gz
210815-19:00:03,893 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/_subject_list_02/_modelestimate0/tstat1.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/02/modelestimate_run_1/tstat1.nii.gz
210815-19:00:03,912 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/_subject_list_02/_modelestimate0/tstat2.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/02/modelestimate_run_1/tstat2.nii.gz
210815-19:00:03,923 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/_subject_list_02/_modelestimate0/tstat3.nii.gz -> /m

210815-19:00:07,267 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 1 jobs ready. Free memory (GB): 6.54/6.74, Free processors: 0/1.
                     Currently running:
                       * _modelestimate0
210815-19:03:04,299 nipype.workflow INFO:
	 [Node] Finished "_modelestimate0".
210815-19:03:05,441 nipype.workflow INFO:
	 [Job 596] Completed (_modelestimate0).
210815-19:03:05,447 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210815-19:03:05,520 nipype.workflow INFO:
	 [Node] Setting-up "_modelestimate1" in "/tmp/tmpilzv5710/first_level_analysis/_subject_list_01/modelestimate/mapflow/_modelestimate1".
210815-19:03:05,527 nipype.workflow INFO:
	 [Node] Running "_modelestimate1" ("nipype.interfaces.fsl.model.FILMGLS"), a CommandLine Interface with command:
film_gls --ms=5 --rn=results --sa --con=/tmp/tmp52pyn86r/first_level_analysis/_subject_list_01/modelgen/mapflow/_modelgen1/run0.con --in=/

210815-19:06:39,847 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/level1design/_subject_list_01/_level1design1/ev_Outlier1_0_11.txt -> /media/paradeisios/UUI/fmri/flanker/output/first_level/level1design/01/level1design_run_2/ev_Outlier1_0_11.txt
210815-19:06:39,850 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/level1design/_subject_list_01/_level1design1/ev_Outlier2_0_12.txt -> /media/paradeisios/UUI/fmri/flanker/output/first_level/level1design/01/level1design_run_2/ev_Outlier2_0_12.txt
210815-19:06:39,855 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/level1design/_subject_list_01/_level1design1/ev_Outlier3_0_13.txt -> /media/paradeisios/UUI/fmri/flanker/output/first_level/level1design/01/level1design_run_2/ev_Outlier3_0_13.txt
210815-19:06:39,861 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/model/_subject_list_01/_modelgen0/run0.m

210815-19:06:40,222 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/_subject_list_01/_modelestimate1/varcope5.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/01/modelestimate_run_2/varcope5.nii.gz
210815-19:06:40,234 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/_subject_list_01/_modelestimate1/varcope6.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/01/modelestimate_run_2/varcope6.nii.gz
210815-19:06:40,251 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/_subject_list_01/_modelestimate0/tstat1.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/01/modelestimate_run_1/tstat1.nii.gz
210815-19:06:40,266 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/first_level/modelestimate/_subject_list_01/_modelestimate0/tstat2.nii.gz -